In [1]:
#Imports
import time
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os
import urllib.request
from urllib.request import urlopen
import pyautogui
from requests.exceptions import ConnectionError
# from pywebcopy import save_webpage

In [2]:

internal_urls = set()
external_urls = set()
def is_url_valid(url):
   #Checks if the url is valid or not
    parsedURL = urlparse(url)
    return bool(parsedURL.netloc) and bool(parsedURL.scheme)

#The above function works as follows. We check if the scheme is present and there is a value in the network location part
# url = "https://umd.edu/virusinfo"
# urlparse(url)
# ParseResult(scheme='https', netloc='umd.edu', path='/virusinfo', 
# params='', query='', fragment='')

#The function gives all urls
def get_all_urls(url):
    urls = set()
    # domain name of the URL without the protocol (umd.edu in this case)
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            #href is empty and we don't need that a element
            continue
        #if the link is not absolute, make it by joining relative to the base
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        #constructing an absolute URL from parsed data
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_url_valid(href):
            #in valid url
            continue
        if href in internal_urls:
            #it is already in the set, so we don't need to add
            continue
        if domain_name not in href:
            #it is an external link. i.e
            # Check if it is already there 
            if href not in external_urls:
#                 print(f"[EXT] External link: {href}")
                external_urls.add(href)
            continue
#         print(".",end=" ")
        print(f"[INT] Internal link: {href}")

        urls.add(href)
        internal_urls.add(href)
    return urls

total_urls_visited = 0
def crawl(url, max_urls=50):
    #Max URL is just to decrease the time if there are a lot of pages.
    #The following code was openly available of github and I found this
    #idea useful to inhibit crawling time
    global total_urls_visited
    total_urls_visited += 1
    try:
        links = get_all_urls(url)
        for link in links:
            if total_urls_visited > max_urls:
                break
            crawl(link, max_urls=max_urls)
    except:
        print("caught")


In [3]:
base_urls = ["https://www.colum.edu/reopening/","https://www.columbusstate.edu/coronavirus/","https://www.concord.edu/coronavirus"]


In [ ]:
for base_url in base_urls:
    internal_urls.add(base_url)
    parsedurl = urlparse(base_url)
    print(parsedurl)
    base_url_main = parsedurl.scheme+"://"+parsedurl.netloc
    print(base_url_main)
    # base_url_text = base_url.split("//",1)[1]
    base_url_text = parsedurl.netloc+parsedurl.path
    print(base_url_text)
    # base_url_text_domain = base_url_text.split("/",1)[0]
    base_url_text_domain = parsedurl.netloc
    print(base_url_text_domain)
    crawl(base_url)
    print("[+] Total External links:", len(external_urls))
    print("[+] Total Internal links:", len(internal_urls))
    print("[+] Total:", len(external_urls) + len(internal_urls))
    count = 0
    last_count = 989874
    immediate_urls = set() #Linked Associated with the current page, https://umd.edu/virusinfo/ in this case

    for i in range(0,4):
        for url in internal_urls:
            if base_url_text in url:
                immediate_urls.add(url)
        count = len(immediate_urls)
        print(count,i)
        if(count == last_count):
            break
        last_count = count
        for immediate_url in immediate_urls:
            try:
                crawl(immediate_url)
                print("[+] NEW Total External links:", len(external_urls))
                print("[+] NEW Total Internal links:", len(internal_urls))
                print("[+] NEW Total:", len(external_urls) + len(internal_urls))
                if len(internal_urls)>1000:
                    break
            except requests.exceptions.ConnectionError:
                continue

    def removeAfterN(yourStr, nth, occurenceOf):
        return occurenceOf.join(yourStr.split(occurenceOf)[:nth])
    
    
    semifinal_immediate_urls = set()

    for immediate_url in immediate_urls:
        stripped_url = removeAfterN(immediate_url,2,"http")


        semifinal_immediate_urls.add(stripped_url)
    #     final_immediate_urls.add(rest)
    #     print(rest)



    for i in semifinal_immediate_urls:
        print(i)
    total_count = len(semifinal_immediate_urls)
    print(total_count)
    
    
    final_immediate_urls = set()
    for semifinal_immediate_url in semifinal_immediate_urls:
        if "tel:" not in semifinal_immediate_url:
            final_immediate_urls.add(semifinal_immediate_url)


    for i in final_immediate_urls:
        print(i)
    total_count = len(final_immediate_urls)
    print(total_count)       
    
    
    
    parent_folder = "Masters" #Change based on folder
    name_of_folder = f'{base_url_text_domain}_Screenshots'
    options = webdriver.ChromeOptions()
    options.headless = True
    driver = webdriver.Chrome(options=options,executable_path='chromedriver.exe') # #Local Path of chrome driver
    if not os.path.exists(f'{parent_folder}\\{name_of_folder}'):
        os.makedirs(f'{parent_folder}\\{name_of_folder}')
    
    
    
    webpage_save = set()
    webpage_save_count = 0
    webpage_cannot_save = 0
    for i in final_immediate_urls:
        if ' ' in i:
            continue
        try:
            code=urlopen(i).getcode()
            webpage_save.add(i)
            webpage_save_count = webpage_save_count+1
            print(f"{code} can_save - {i}")
        except (urllib.error.HTTPError,urllib.error.URLError) as e:
            print("discarded")
            webpage_cannot_save = webpage_cannot_save+1
            continue

    print(f'Webpages save as HTML :{webpage_save_count}')
    print(f'Webpages unable to save as HTML :{webpage_cannot_save}')

    
    pdf_count = 0
    pdf_urls = set()
    for webpage in webpage_save:
        try:
            if webpage[-4:] == ".pdf":
                url = webpage
                file_name = url.replace(base_url,'')
                file_name = file_name.replace('/','')
                if file_name == '':
                    file_name = "Home"
                pdf_urls.add(webpage)
                print(webpage)
                r = requests.get(url, stream=True)
                with open(f'{parent_folder}\\{name_of_folder}\\{file_name}.pdf', 'wb') as fd:
                    for chunk in r.iter_content(2000):
                        fd.write(chunk)
                pdf_count = pdf_count + 1
        except:
            print("caught")

    print(f"Pdfs saved: {pdf_count}")
    
    
    
    final_immediate_urls = final_immediate_urls - pdf_urls
    webpage_save = webpage_save - pdf_urls
    web_save_count = len(webpage_save)
    total_count = len(final_immediate_urls)
    print(total_count) 
    
    
    
    final_webpage_save_count = webpage_save_count - pdf_count
    print(final_webpage_save_count)
    
    
    webpage_save_counter_loop = 0
    for webpage in webpage_save:
        try:
            url = webpage
            file_name = url.replace(base_url,'')
            file_name = file_name.replace('/','')
            if file_name == '':
                    file_name = "Home"
            urllib.request.urlretrieve (webpage, f"{parent_folder}\\{name_of_folder}\\{file_name}.html")
            webpage_save_counter_loop = webpage_save_counter_loop+1
            print(f'{webpage_save_counter_loop} of {final_webpage_save_count} - {webpage}')
        except:
            print("caught")

    print("Saving Completed")
    
    
    count = 0
    for i in final_immediate_urls:
        try:
            url = i
            file_name = url.replace(base_url,'')
            file_name = file_name.replace('/','')
            if file_name == '':
                    file_name = "Home"
            count = count + 1
            print(f"{count} of {total_count}")
            print(f'Visiting {base_url_text}/{file_name}')
            print(f'...Taking a screenshot')
            driver.get(url)
            S = lambda X: driver.execute_script('return document.body.parentNode.scroll'+X)
            driver.set_window_size(S('Width'),S('Height')) # May need manual adjustment                                                                                                                
            driver.find_element_by_tag_name('body').screenshot(f'{parent_folder}\\{name_of_folder}\\{file_name}.png')
            print(f'Screenshot of {file_name} page taken! \n')
        except:
            print("caught")
    #     print("Saving HTML")
    #     pyautogui.hotkey('ctrl', 's')
    #     time.sleep(1)
    #     pyautogui.typewrite(f'{file_name}.html')
    #     pyautogui.hotkey('enter')
    driver.quit()

    print(f'Task Completed! Files stored in the {parent_folder}\\{name_of_folder} Folder')

ParseResult(scheme='https', netloc='www.colum.edu', path='/reopening/', params='', query='', fragment='')
https://www.colum.edu
www.colum.edu/reopening/
www.colum.edu
[INT] Internal link: https://www.colum.edu
[INT] Internal link: https://www.colum.edu/academics/index.html
[INT] Internal link: https://www.colum.edu/academics/majors-and-programs.html
[INT] Internal link: https://www.colum.edu/academics/graduate-studies/index.html
[INT] Internal link: https://www.colum.edu/academics/columbia-core-curriculum/index.html
[INT] Internal link: https://www.colum.edu/academics/columbia-core-curriculum/honors-program.html
[INT] Internal link: https://www.colum.edu/academics/academic-calendar.html
[INT] Internal link: https://www.colum.edu/academics/faculty/index.html
[INT] Internal link: https://www.colum.edu/academics/alumni/index.html
[INT] Internal link: https://www.colum.edu/academics/alumni/demo-magazine/index
[INT] Internal link: https://www.colum.edu/academics/alumni/notable/index.html
[I

[INT] Internal link: http://www.colum.edu/index.html
[INT] Internal link: http://www.colum.edu/curatorial-proposals/index.html
[INT] Internal link: http://www.colum.edu/opportunities.html
[INT] Internal link: http://www.colum.edu/event-spaces/index.html
[INT] Internal link: http://www.colum.edu/contact.html
[INT] Internal link: https://www.colum.edu/index.html
[INT] Internal link: http://www.colum.edu/glass-curtain-gallery/index.html
[INT] Internal link: http://www.colum.edu/c33-gallery/index.html
[INT] Internal link: http://www.colum.edu/hokin-gallery/index.html
[INT] Internal link: http://www.colum.edu/calendar/index.html
[INT] Internal link: http://www.colum.edu/Tactical_Pages/forms/Web_Author_Contact.php
[INT] Internal link: http://www.colum.edu/academics/index.html
[INT] Internal link: http://www.colum.edu/academics/majors-and-programs.html
[INT] Internal link: http://www.colum.edu/academics/graduate-studies/index.html
[INT] Internal link: http://www.colum.edu/academics/columbia-c

[INT] Internal link: https://www.colum.edu/admissions/visit-us/virtual-tours
[INT] Internal link: https://www.colum.edu/admissions/visit-us/
[INT] Internal link: https://www.colum.edu/admissions/requirements/international/graduate.html
[INT] Internal link: https://www.colum.edu/admissions/visit-us/getting-to-campus.html
[INT] Internal link: https://www.colum.edu/admissions/visit-us/virtual.html
[INT] Internal link: https://www.colum.edu/admissions/visit-us/virtual-tours.html
[INT] Internal link: https://www.colum.edu/admissions/additional-information/bfa-requirements.html
[INT] Internal link: https://www.colum.edu/admissions/additional-information/timeline/index.html
[INT] Internal link: https://www.colum.edu/admissions/additional-information/timeline/graduate.html
[INT] Internal link: https://www.colum.edu/admissions/additional-information/transfer-planning-tools.html
[INT] Internal link: https://www.colum.edu/admissions/additional-information/first-generation-students.html
[INT] Inte

[INT] Internal link: https://www.colum.edu/student-financial-services/how-to-pay/sap-undergraduate.html
[INT] Internal link: https://www.colum.edu/student-financial-services/how-to-pay/sap-graduate.html
[INT] Internal link: https://www.colum.edu/student-financial-services/how-to-pay/overborrowing-reaffirmation.html
[INT] Internal link: https://www.colum.edu/student-financial-services/where-to-start/drug-disclosure.html
[INT] Internal link: https://www.colum.edu/student-financial-services/scholarships/graduate-assistantship.html
[INT] Internal link: https://www.colum.edu/student-financial-services/how-to-pay/federal-loan-information.html
[INT] Internal link: https://www.colum.edu/student-financial-services/how-to-pay/private-loans.html
[INT] Internal link: https://www.colum.edu/student-financial-services/where-to-start/what-type-of-student-are-you/admitted-student.html
[INT] Internal link: https://www.colum.edu/student-financial-services/where-to-start/what-type-of-student-are-you/curre

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


...

[INT] Internal link: https://www.columbusstate.edu/coronavirus/Communication_on_Instruction_3-13-2020.pdf
[INT] Internal link: http://text.usg.edu/tt/https://www.columbusstate.edu/coronavirus/updates/3-13-communication-on-instruction.php
[+] NEW Total External links: 393
[+] NEW Total Internal links: 810
[+] NEW Total: 1203
caught
[+] NEW Total External links: 393
[+] NEW Total Internal links: 810
[+] NEW Total: 1203
caught
[+] NEW Total External links: 393
[+] NEW Total Internal links: 810
[+] NEW Total: 1203
[INT] Internal link: http://text.usg.edu/tt/https://www.columbusstate.edu/coronavirus/resources/employee-resources/guidance-protecting-education-records.php
[+] NEW Total External links: 395
[+] NEW Total Internal links: 811
[+] NEW Total: 1206
[INT] Internal link: http://text.usg.edu/tt/https://www.columbusstate.edu/coronavirus/updates/4-10-graduation-update-message.php
[+] NEW Total External links: 397
[+] NEW Total Internal links: 812
[+] NEW Total: 1209
[INT] Internal link: h

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[+] NEW Total External links: 399
[+] NEW Total Internal links: 829
[+] NEW Total: 1228
[+] NEW Total External links: 399
[+] NEW Total Internal links: 829
[+] NEW Total: 1228
[+] NEW Total External links: 399
[+] NEW Total Internal links: 829
[+] NEW Total: 1228
[+] NEW Total External links: 399
[+] NEW Total Internal links: 829
[+] NEW Total: 1228
caught
[+] NEW Total External links: 399
[+] NEW Total Internal links: 829
[+] NEW Total: 1228
caught
[+] NEW Total External links: 399
[+] NEW Total Internal links: 829
[+] NEW Total: 1228
caught
[+] NEW Total External links: 399
[+] NEW Total Internal links: 829
[+] NEW Total: 1228


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[+] NEW Total External links: 399
[+] NEW Total Internal links: 829
[+] NEW Total: 1228
caught
[+] NEW Total External links: 399
[+] NEW Total Internal links: 829
[+] NEW Total: 1228
caught
[+] NEW Total External links: 399
[+] NEW Total Internal links: 829
[+] NEW Total: 1228
[+] NEW Total External links: 399
[+] NEW Total Internal links: 829
[+] NEW Total: 1228
caught
[+] NEW Total External links: 399
[+] NEW Total Internal links: 829
[+] NEW Total: 1228
[+] NEW Total External links: 399
[+] NEW Total Internal links: 829
[+] NEW Total: 1228
[+] NEW Total External links: 399
[+] NEW Total Internal links: 829
[+] NEW Total: 1228
[+] NEW Total External links: 399
[+] NEW Total Internal links: 829
[+] NEW Total: 1228
caught
[+] NEW Total External links: 399
[+] NEW Total Internal links: 829
[+] NEW Total: 1228
